# CIFAR-100: Full Lab — Account setup, Train → W&B → Deploy on Hugging Face Spaces
**Colab-ready hands-on lab notebook**

This notebook combines:
- Step-by-step instructions to create a **Hugging Face** account and **access token**,
- Secure token input in Colab using `getpass` (token is not printed),
- **W&B** authentication,
- Training a `resnet18` on **CIFAR-100**, logging to W&B and saving model as an artifact,
- A Gradio app (`app.py`) that downloads the model artifact from W&B at startup,
- A script to create & push a **Hugging Face Space** (Gradio) from Colab using `huggingface_hub`.

**Security note:** Never commit your tokens/API keys to public repos. Use Colab `getpass` to keep tokens secret and use HF Space Secrets for W&B keys.


In [1]:
# DO NOT hardcode API keys! Use environment variables or getpass instead.
import os
from getpass import getpass

# Option 1: Load from environment (recommended)
wandb_key = os.environ.get('WANDB_API_KEY')
if not wandb_key:
    # Option 2: Prompt securely if not set
    wandb_key = getpass('Enter W&B API key: ')
    os.environ['WANDB_API_KEY'] = wandb_key
else:
    print("✅ W&B API key loaded from environment")

In [24]:
from getpass import getpass
os.environ['HF_USER'] = input('Enter HF username: ')
os.environ['SPACE_NAME'] = input('Enter space name: ')
os.environ['HF_TOKEN'] = getpass('Enter HF token: ') 

In [3]:
#@title 🔍 Environment Validation (run before starting lab)
import torch, os

print("---- Environment Validation ----")

# Check GPU availability
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

# Check required env vars
for var in ["HF_TOKEN", "HF_USER", "SPACE_NAME"]:
    val = os.environ.get(var)
    print(f"{var}:", " SET" if val else "NOT SET")

# Check if W&B API key configured (login handled separately)
wandb_key = os.environ.get("WANDB_API_KEY")
print("WANDB_API_KEY:", " SET" if wandb_key else "⚠️ Not set (you'll log in interactively)")

# Check if dependencies installed
try:
    import wandb, gradio, huggingface_hub
    print("Dependencies:  wandb, gradio, huggingface_hub imported successfully")
except Exception as e:
    print("Dependencies: missing - please run install cell first")
    print(e)

print("---------------------------------")


---- Environment Validation ----
GPU available: False
HF_TOKEN:  SET
HF_USER:  SET
SPACE_NAME:  SET
WANDB_API_KEY:  SET


/home/chakri/Documents/Projects/Chicken-Disease-Classification/chicken/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/home/chakri/Documents/Projects/Chicken-Disease-Classification/chicken/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/chakri/Documents/Projects/Chicken-Disease-Classification/chicken/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stabl

Dependencies:  wandb, gradio, huggingface_hub imported successfully
---------------------------------


In [ ]:
#@title 1) Install dependencies (run once)
# Installs pytorch (with CUDA if available), W&B, Gradio and HF tooling.
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install -q wandb gradio huggingface_hub git-lfs

## 2) Hugging Face account & token (student instructions)

**If you don't have a Hugging Face account:**
1. Go to https://huggingface.co and click **Sign up**. Verify email.
2. Choose a username (used as `HF_USER`).

**Create an Access Token:**
1. Go to https://huggingface.co/settings/tokens
2. Click **New token**. Name it (e.g., `colab-space-token`) and select scope `repo` (read & write).
3. Click **Create** and copy the token now — you won't see it again.

**Important:** Keep the token secret. Use the next cell to set it securely in Colab (the token will not be printed).

In [ ]:
#@title 3) Securely set your Hugging Face token, username, and desired Space name
from getpass import getpass
import os

print("Paste your Hugging Face token when prompted. It will be hidden.")
hf_token = getpass("Hugging Face token: ")
os.environ['HF_TOKEN'] = hf_token

# Edit these values (do NOT put the token here)
hf_user = input("Enter your Hugging Face username (e.g. 'alice'): ").strip()
space_name = input("Enter desired Space name (e.g. 'cifar100-demo-space'): ").strip()

os.environ['HF_USER'] = hf_user
os.environ['SPACE_NAME'] = space_name

print("HF_TOKEN stored in runtime (hidden). HF_USER and SPACE_NAME saved in environment variables.")

Paste your Hugging Face token when prompted. It will be hidden.


In [4]:
#@title 4) Authenticate Weights & Biases (W&B)
import wandb
print("Follow the prompt to authenticate W&B (this opens an input box).")
wandb.login()


Follow the prompt to authenticate W&B (this opens an input box).


wandb: Currently logged in as: 112201022 (ir2023) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## 5) Create `train.py` — training + W&B artifact logging

This script trains ResNet18 on CIFAR-100, logs metrics to W&B, and saves the best model as a W&B artifact.


In [5]:
%%bash
cat > train.py <<'PY'
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import wandb
from torch.utils.data import DataLoader
from torchvision.models import resnet18

def parse_args():
    p = argparse.ArgumentParser()
    p.add_argument("--project", type=str, default="cifar100-hf-demo")
    p.add_argument("--entity", type=str, default=None)
    p.add_argument("--epochs", type=int, default=5)
    p.add_argument("--batch-size", type=int, default=128)
    p.add_argument("--lr", type=float, default=0.01)
    return p.parse_args()

def get_dataloaders(batch_size):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409),
                             (0.2673, 0.2564, 0.2762)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409),
                             (0.2673, 0.2564, 0.2762)),
    ])
    trainset = torchvision.datasets.CIFAR100(root="./data", train=True, download=True, transform=transform_train)
    testset  = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform_test)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    testloader  = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    return trainloader, testloader

def train_one_epoch(model, device, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return running_loss / total, 100. * correct / total

def evaluate(model, device, loader, criterion):
    model.eval()
    loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            l = criterion(outputs, targets)
            loss += l.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return loss/total, 100.*correct/total

def main():
    args = parse_args()
    wandb.init(project=args.project, entity=args.entity, config=vars(args))
    cfg = wandb.config

    device = "cuda" if torch.cuda.is_available() else "cpu"
    trainloader, testloader = get_dataloaders(cfg.batch_size)

    model = resnet18(num_classes=100)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=cfg.lr, momentum=0.9, weight_decay=5e-4)

    best_acc = 0.0
    for epoch in range(cfg.epochs):
        train_loss, train_acc = train_one_epoch(model, device, trainloader, optimizer, criterion)
        test_loss, test_acc = evaluate(model, device, testloader, criterion)
        wandb.log({"epoch": epoch+1, "train_loss": train_loss, "train_acc": train_acc,
                   "test_loss": test_loss, "test_acc": test_acc})
        print(f"Epoch {epoch+1}: train_acc={train_acc:.2f} test_acc={test_acc:.2f}")
        if test_acc > best_acc:
            best_acc = test_acc
            os.makedirs("outputs", exist_ok=True)
            torch.save(model.state_dict(), "outputs/model.pt")
            # log artifact
            artifact = wandb.Artifact("resnet18-cifar100", type="model", metadata={"test_acc": best_acc})
            artifact.add_file("outputs/model.pt")
            wandb.log_artifact(artifact)
    print("Best test acc:", best_acc)

if __name__ == "__main__":
    main()
PY


In [6]:
#@title 6) Run training (edit the --entity to your W&B username/team)
# Keep epochs small for demo (e.g., 3-5). Increase for better accuracy.
!python train.py --project cifar100-hf-demo --entity 'ir2023' --epochs 3 --batch-size 128


wandb: Currently logged in as: 112201022 (ir2023) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ setting up run cg359xeb (0.0s)
wandb: ⣻ setting up run cg359xeb (0.0s)
wandb: ⣽ setting up run cg359xeb (0.0s)
wandb: ⣽ setting up run cg359xeb (0.0s)
wandb: ⣾ setting up run cg359xeb (0.0s)
wandb: ⣾ setting up run cg359xeb (0.0s)
wandb: Tracking run with wandb version 0.22.2
wandb: Run data is saved locally in /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-10/wandb/run-20251022_004042-cg359xeb
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run wise-sky-18
wandb: ⭐️ View project at https://wandb.ai/ir2023/cifar100-hf-demo
wandb: 🚀 View run at https://wandb.ai/ir2023/cifar100-hf-demo/runs/cg359xeb
wandb: Tracking run with wandb version 0.22.2
wandb: Run data is saved locally in /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-10/wandb/run

## 7) Download model artifact from W&B (if you didn't run training here)
This downloads the latest logged artifact into `outputs/` so the Gradio app can load it.

In [7]:
os.environ['WANDB_ENTITY']= "ir2023"

In [8]:
%%bash
python - <<'PY'
import wandb, os, sys
ENTITY = os.environ.get("WANDB_ENTITY") or "ir2023"   # <-- edit if not set
PROJECT = "cifar100-hf-demo"
ARTIFACT = "resnet18-cifar100:latest"
api = wandb.Api()
try:
    artifact = api.artifact(f"{ENTITY}/{PROJECT}/{ARTIFACT}")
    artifact.download(root="outputs")
    print("Downloaded artifact to outputs/")
except Exception as e:
    print("Failed to download artifact:", e)
    sys.exit(1)
PY


wandb:   1 of 1 files downloaded.  


Downloaded artifact to outputs/


## 8) Create Gradio app (`app.py`) that downloads the model from W&B at startup

This approach avoids committing large model files into the Space repository. The Space must have `WANDB_API_KEY` set as a secret in its settings.


In [33]:
%%bash
cat > app.py <<'PY'
import os, time, io, shutil
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
import gradio as gr

MODEL_PATH = "outputs/model.pt"

# If model not present, try download via W&B (requires WANDB_API_KEY secret in Space or env)
if not os.path.exists(MODEL_PATH):
    try:
        import wandb
        wandb_api_key = os.environ.get("WANDB_API_KEY")
        if wandb_api_key:
            wandb.login(key=wandb_api_key)
            api = wandb.Api()
            artifact_path = os.environ.get("WANDB_ARTIFACT", "ir2023/cifar100-hf-demo/resnet18-cifar100:latest")
            print(f"Downloading artifact: {artifact_path}")
            artifact = api.artifact(artifact_path)
            # Remove outputs dir if exists, then download (avoids replace parameter issue)
            if os.path.exists("outputs"):
                shutil.rmtree("outputs")
            artifact.download(root="outputs")
            print("Downloaded model via W&B artifact successfully.")
        else:
            print("WANDB_API_KEY not set; cannot download artifact.")
    except Exception as e:
        print(f"Error downloading artifact via W&B: {e}")
        import traceback
        traceback.print_exc()

device = "cuda" if torch.cuda.is_available() else "cpu"
model = resnet18(num_classes=100)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409),(0.2673,0.2564,0.2762))
])

def predict_image(img):
    start = time.time()
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(x)
        probs = torch.nn.functional.softmax(out, dim=1)
        conf, idx = probs.max(1)
        class_idx = int(idx.item())
        conf_val = float(conf.item())
    latency = (time.time() - start) * 1000.0
    return {"class_idx": class_idx, "confidence": round(conf_val,4), "latency_ms": round(latency,2)}

iface = gr.Interface(fn=predict_image, inputs=gr.Image(type="pil"), outputs="json", title="CIFAR-100 demo")
if __name__ == "__main__":
    iface.launch(server_name="0.0.0.0", server_port=7860)
PY

In [34]:
%%bash
cat > requirements.txt <<'REQ'
torch
torchvision
gradio
Pillow
wandb
huggingface_hub
git-lfs
REQ


## 9) Create & push a Hugging Face Space from Colab (uses HF_TOKEN set earlier via getpass)

This will:
- create the Space repo (if it doesn't exist),
- push `app.py` and `requirements.txt` to the Space,
- not include the heavy model file (the app downloads it from W&B at runtime).


In [35]:
#@title 🔍 Check HuggingFace Token Permissions
import os
from huggingface_hub import HfApi

token = os.environ.get("HF_TOKEN")
if not token:
    print("❌ HF_TOKEN not set!")
else:
    try:
        api = HfApi(token=token)
        user_info = api.whoami()
        print("✅ Token is valid!")
        print(f"Username: {user_info['name']}")
        print(f"Token type: {user_info.get('auth', {}).get('type', 'Unknown')}")
        
        # Check if token has write permissions
        scopes = user_info.get('auth', {}).get('accessToken', {}).get('role', 'read')
        print(f"Token role/scope: {scopes}")
        
        if scopes == 'read':
            print("\n⚠️  WARNING: Your token has READ-ONLY permissions!")
            print("   You CANNOT create Spaces with this token.")
            print("\n🔧 To fix:")
            print("   1. Go to: https://huggingface.co/settings/tokens")
            print("   2. Click 'New token'")
            print("   3. Select 'Write' permission (NOT 'Read')")
            print("   4. Copy the new token")
            print("   5. Re-run Cell 3 and paste the NEW token")
        elif scopes == 'write':
            print("\n✅ Token has WRITE permissions - ready to create Spaces!")
        else:
            print(f"\n⚠️  Unknown permission level: {scopes}")
            
    except Exception as e:
        print(f"❌ Error checking token: {e}")
        print("   The token may be invalid or revoked.")

✅ Token is valid!
Username: Chakradharreddy3237
Token type: access_token
Token role/scope: fineGrained

⚠️  Unknown permission level: fineGrained


In [36]:
#@title 🔍 Check Fine-Grained Token Permissions (Detailed)
import os
from huggingface_hub import HfApi

token = os.environ.get("HF_TOKEN")
if not token:
    print("❌ HF_TOKEN not set!")
else:
    try:
        api = HfApi(token=token)
        user_info = api.whoami()
        
        print("=" * 70)
        print("HUGGING FACE TOKEN INFORMATION")
        print("=" * 70)
        print(f"Username: {user_info['name']}")
        print(f"Token type: {user_info.get('auth', {}).get('type', 'Unknown')}")
        
        # For fine-grained tokens, check permissions
        auth_info = user_info.get('auth', {})
        access_token = auth_info.get('accessToken', {})
        
        print(f"\nToken role: {access_token.get('role', 'N/A')}")
        
        # Check if repo.write permission exists
        permissions = access_token.get('permissions', {})
        print(f"\nPermissions:")
        for key, value in permissions.items():
            print(f"  - {key}: {value}")
        
        # Check if we can create repos/spaces
        repo_write = permissions.get('repo.write', False) or permissions.get('repo', {}).get('write', False)
        
        print("\n" + "=" * 70)
        if repo_write or str(repo_write) == 'write':
            print("✅ SUCCESS: Token has WRITE permissions for repositories!")
            print("   You can create Spaces with this token.")
        else:
            print("❌ ERROR: Token does NOT have WRITE permissions!")
            print("   Current permissions won't allow creating Spaces.")
            print("\n🔧 FIX:")
            print("   1. Go to: https://huggingface.co/settings/tokens")
            print("   2. Delete this token or create a NEW token")
            print("   3. When creating, ensure you select:")
            print("      ☑️  'Write access to repos'  OR")
            print("      ☑️  'Make repo' permission")
            print("   4. Copy the new token")
            print("   5. Re-run Cell 3 and paste the NEW token")
        print("=" * 70)
            
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()

HUGGING FACE TOKEN INFORMATION
Username: Chakradharreddy3237
Token type: access_token

Token role: fineGrained

Permissions:

❌ ERROR: Token does NOT have WRITE permissions!
   Current permissions won't allow creating Spaces.

🔧 FIX:
   1. Go to: https://huggingface.co/settings/tokens
   2. Delete this token or create a NEW token
   3. When creating, ensure you select:
      ☑️  'Write access to repos'  OR
      ☑️  'Make repo' permission
   4. Copy the new token
   5. Re-run Cell 3 and paste the NEW token


In [37]:
%%bash
set -e
# prepare local repo
rm -rf hf_space || true
mkdir hf_space
cp app.py requirements.txt hf_space/
cd hf_space

git init
git config user.email "112201022@smail.iitpkd.ac.in"
git config user.name "ChakradharReddy3237"
# git lfs install  # Commented out - not needed

python - <<'PY'
from huggingface_hub import HfApi, Repository
import os, sys
token = os.environ.get("HF_TOKEN")
user = os.environ.get("HF_USER")
space = os.environ.get("SPACE_NAME")
if not token or not user or not space:
    print("HF_TOKEN, HF_USER or SPACE_NAME not set. Aborting.")
    sys.exit(1)
api = HfApi(token=token)

repo_id = f"{user}/{space}"
repo_url = api.create_repo(repo_id=repo_id, repo_type="space",
            space_sdk="gradio",
            exist_ok=True)
print("Repo URL:", repo_url)

api.upload_folder(
    folder_path=".",
    repo_id=repo_id,
    repo_type="space",
    commit_message="Initial commit: CIFAR-100 Gradio app (no model)"
)

print("Pushed to:", repo_url)

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
o change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>


Initialized empty Git repository in /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-10/hf_space/.git/
-10/hf_space/.git/


bash: line 37: warning: here-document at line 13 delimited by end-of-file (wanted `PY')


Repo URL: https://huggingface.co/spaces/Chakradharreddy3237/mlops_class10_project


No files have been modified since last commit. Skipping to prevent empty commit.


Pushed to: https://huggingface.co/spaces/Chakradharreddy3237/mlops_class10_project


## ✅ Deployment Successful!

Your Space has been created at: **https://huggingface.co/spaces/Chakradharreddy3237/mlops_class10_project**

### Next: Configure Secrets

The Space needs your W&B API key to download the model. Follow these steps:

1. **Open Space Settings**: https://huggingface.co/spaces/Chakradharreddy3237/mlops_class10_project/settings

2. **Click "Repository secrets"** in the left sidebar

3. **Add these secrets:**
   - **Name**: `WANDB_API_KEY`
     - **Value**: Your W&B API key (same one you used in Cell 2)
   
   - **Name**: `WANDB_ARTIFACT`
     - **Value**: `ir2023/cifar100-hf-demo/resnet18-cifar100:latest`

4. **Click "Save"** for each secret

5. **Wait for the Space to rebuild** (it will automatically restart)

6. **Visit your Space**: https://huggingface.co/spaces/Chakradharreddy3237/mlops_class10_project

7. **Test it**: Upload a CIFAR-100 image and check if predictions work!

In [38]:
#@title 📋 Get W&B API Key (for copying to Space secrets)
import os
import webbrowser

wandb_key = os.environ.get('WANDB_API_KEY')
if wandb_key:
    print("=" * 70)
    print("YOUR W&B API KEY (copy this to Space secrets):")
    print("=" * 70)
    print(wandb_key)
    print("=" * 70)
    print("\n✅ Copy the key above")
    print("\n📌 Now open Space settings:")
    print("   https://huggingface.co/spaces/Chakradharreddy3237/mlops_class10_project/settings")
    print("\n🔧 Steps:")
    print("   1. Click 'Repository secrets' in left sidebar")
    print("   2. Add secret: WANDB_API_KEY = <paste key above>")
    print("   3. Add secret: WANDB_ARTIFACT = ir2023/cifar100-hf-demo/resnet18-cifar100:latest")
    print("   4. Save and wait for Space to rebuild")
    print("   5. Visit: https://huggingface.co/spaces/Chakradharreddy3237/mlops_class10_project")
else:
    print("❌ W&B API key not found. Re-run Cell 2 to set it.")

YOUR W&B API KEY (copy this to Space secrets):
fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f

✅ Copy the key above

📌 Now open Space settings:
   https://huggingface.co/spaces/Chakradharreddy3237/mlops_class10_project/settings

🔧 Steps:
   1. Click 'Repository secrets' in left sidebar
   2. Add secret: WANDB_API_KEY = <paste key above>
   3. Add secret: WANDB_ARTIFACT = ir2023/cifar100-hf-demo/resnet18-cifar100:latest
   4. Save and wait for Space to rebuild
   5. Visit: https://huggingface.co/spaces/Chakradharreddy3237/mlops_class10_project


## 10) Adding W&B API key as a secret in the Hugging Face Space

After pushing the Space, do:
1. Visit `https://huggingface.co/spaces/<HF_USER>/<SPACE_NAME>`.
2. Click **Settings → Secrets**.
3. Add a secret named `WANDB_API_KEY` and paste your W&B API key (keep secret).
4. Optionally set `WANDB_ARTIFACT` to the artifact path, e.g. `your_wandb_entity/cifar100-hf-demo/resnet18-cifar100:latest`.
5. Rebuild the Space (there's a "Rebuild" button) so the app can download the artifact at startup.


## 11) Test the deployed Space
Open the Space URL `https://huggingface.co/spaces/<HF_USER>/<SPACE_NAME>`.
- Check the Space logs (Settings → Logs) if the model download or runtime fails.
- Common issues: missing secret, wrong artifact name, package install errors (pin versions in requirements.txt).


## 12) Instructor tips and cleanup
- Keep epochs small during demos.
- Instructor can host one shared Space and W&B project to avoid per-student pushes.
- After course, remove the Space or set it to private if not needed.
- Revoke tokens if accidentally exposed: Hugging Face (Settings → Access Tokens), W&B (Settings → API Keys).
